## Libraries

In [1]:
import chatbot
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import json
from dotenv import load_dotenv
from openai import OpenAI
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import time

load_dotenv()
client = OpenAI()

**Load index and metadata**

In [2]:
chatbot.set_vector_dir("data/05_vectorized/large/c400_40")
index, metadata = chatbot.load_faiss_index(chatbot.VECTOR_DIR)
bm25=chatbot.build_bm25(metadata)
print(f"Index and metadata loaded at {chatbot.VECTOR_DIR}")

Index and metadata loaded at data/05_vectorized/large/c400_40


**Loading Evaluation Dataset**

In [6]:
with open("evaluation/evaluation_dataset_v2.json", "r", encoding="utf-8") as f:
    data = json.load(f)

def json_to_documents(data: dict):
    documents = []

    for source, content in data.items():
        qa_list = content.get("perguntas_respostas", [])

        for item in qa_list:
            documents.append({
                "question": item.get("pergunta", ""),
                "answer": item.get("resposta", ""),
                "source": source
            })

    return documents

eval_dataset = json_to_documents(data)

**Chatbot Responses**

In [7]:
from tqdm import tqdm

def populate_eval_dataset(eval_dataset, index, metadata, bm25, 
                          model="gpt-4o-mini", k=20, top_k=5, 
                          weight_dense=0.6, weight_sparse=0.4, rerank=False):
    """
    Fill the 'answer' and 'contexts' fields in the evaluation dataset
    by calling your bot function.

    Args:
        eval_dataset (list of dicts): each dict must have 'question' and optionally 'source'
        index, metadata, bm25: your RAG components
        model, k, top_k, weight_dense, weight_sparse, rerank: bot settings

    Returns:
        list of dicts: same dataset with 'answer' and 'contexts' populated
    """
    updated_dataset = []

    for sample in tqdm(eval_dataset, desc="Generating bot answers"):
        question = sample["question"]
        # Optional: use sample['source'] if needed by retriever
        source = sample.get("source", "")

        # Call your existing bot
        generated_answer, context_chunks = chatbot.answer(
            user_query=question,
            index=index,
            metadata=metadata,
            bm25=bm25,
            model=model,
            k=k,
            top_k=top_k,
            weight_dense=weight_dense,
            weight_sparse=weight_sparse,
            rerank=rerank
        )

        # Save results back into sample
        sample["bot_answer"] = generated_answer
        sample["contexts"] = [c["content"] for c in context_chunks]

        updated_dataset.append(sample)

    return updated_dataset


In [8]:
eval_dataset_filled_c400_40 = populate_eval_dataset(
    eval_dataset, index, metadata, bm25,
    model="gpt-4o-mini",
    k=10,
    top_k=10,
    weight_dense=0.5,
    weight_sparse=0.5,
    rerank=False
)


Generating bot answers:   3%|▌                   | 1/35 [00:04<02:43,  4.82s/it]


Assistente: O Alentejo 2030 é um programa regional aprovado pela Comissão Europeia para o período de programação 2021-2027, que visa promover a competitividade da economia, a sustentabilidade ambiental e a valorização do território e das pessoas na região do Alentejo. Este programa tem como objetivo operacionalizar a estratégia de desenvolvimento da região e as estratégias sub-regionais das Comunidades Intermunicipais. O Alentejo 2030 também se propõe a transformar vidas, impulsionando a inovação e o crescimento na região, através da disponibilização de fundos europeus. 

A sua implementação resulta de um processo iniciado em 2018, que incluiu a elaboração de um Contributo Estratégico Regional e a definição de uma visão estratégica com seis objetivos para enfrentar desafios como o constrangimento demográfico. 

Fonte: alentejo.portugal2030.pt.


Generating bot answers:   6%|█▏                  | 2/35 [00:09<02:32,  4.63s/it]


Assistente: Alguns dos eventos programados para o Alentejo 2030 incluem:

1. **Workshop – Prospetivas e Políticas Públicas** - 15 de dezembro de 2025.
2. **Conferência e Espetáculo | “Inteligência Artificial”** - 27 de novembro de 2025.
3. **Concerto – “Ecoando a voz do Alentejo”** - 16 de novembro de 2025, na Vidigueira.
4. **Como os Fundos Europeus potenciam a Inovação Social** - 5 de junho de 2025.
5. **Sessão de apresentação das Plataformas Regionais** - 12 de junho de 2025, integrada na Estratégia de Desenvolvimento Regional EREI 2030.

Esses eventos visam promover a reflexão e o desenvolvimento em diversas áreas, como inovação social e políticas públicas. 

Fonte: alentejo.portugal2030.pt.


Generating bot answers:   9%|█▋                  | 3/35 [00:12<02:14,  4.20s/it]

...


Assistente: Os principais eventos relacionados ao Portugal 2030 incluem:

1. **Comité de Acompanhamento** - Reuniões periódicas do Comité de Acompanhamento do Lisboa 2030, onde são discutidos os progressos e as metas do programa.

2. **Sessões de Divulgação** - Eventos como a sessão de divulgação conjunta do Centro, Alentejo e Lisboa, que abordam avisos inter-regionais e oportunidades de financiamento.

3. **Reuniões de Acompanhamento** - Encontros que visam apresentar a taxa de execução dos programas e discutir a implementação de iniciativas.

4. **Workshops e Conferências** - Ações formativas e informativas que promovem a capacitação e o conhecimento sobre os fundos e programas disponíveis.

Esses eventos são fundamentais para promover a interação entre os diversos stakeholders, facilitar o acesso à informação e fomentar a colaboração no âmbito do Portugal 2030.

Fontes: Lisboa2030, Portugal2030.


Generating bot answers:  97%|██████████████████▍| 34/35 [02:37<00:05,  5.34s/it]


Assistente: Para obter apoio técnico sobre o Balcão dos Fundos, você pode utilizar as seguintes opções:

1. **Email**: Envie um email para linhadosfundos@linhadosfundos.pt, onde poderá esclarecer as suas dúvidas.

2. **Telefone**: Ligue para o número 800 10 35 10, que é gratuito para chamadas de Portugal, ou para 00351 300 00 39 87, se estiver a ligar do estrangeiro (custo de chamada para a rede fixa de Portugal).

3. **Chat Online**: Aceda ao site da Linha dos Fundos para utilizar o chat, disponível todos os dias úteis entre as 9h e as 18h.

4. **Videochamada**: É possível agendar uma videochamada para apoio ao registo no Balcão dos Fundos.

Essas opções garantem que você tenha acesso a informações e suporte técnico sobre o Balcão dos Fundos e outras plataformas relacionadas.

Fontes: Portugal2030.


Generating bot answers: 100%|███████████████████| 35/35 [02:43<00:00,  4.68s/it]


Assistente: O projeto CustomNiTi é uma iniciativa cofinanciada pelo Programa COMPETE 2030, que visa revolucionar o setor de dispositivos médicos, especificamente na área da ortopedia. O projeto foca no desenvolvimento de placas e distratores ósseos personalizados, com o objetivo de melhorar a eficiência e o conforto no tratamento de fraturas.

As principais características e objetivos do projeto incluem:

1. **Inovação** - O CustomNiTi representa um avanço significativo na medicina moderna, criando soluções clínicas eficazes e económicas.

2. **Capacidade Industrial** - O projeto fortalece a capacidade industrial e competitiva da CPIMC (Customize Portugal), posicionando-a como uma referência nacional no fabrico de dispositivos médicos personalizados e inovadores.

3. **Impacto Clínico** - As soluções desenvolvidas têm um elevado potencial económico e um impacto positivo na saúde dos pacientes.

4. **Imagem de Inovação** - O projeto contribui para reforçar a imagem de Portugal como um 

In [9]:
with open("evaluation/eval_dataset_filled_c400_40.json", "w", encoding="utf-8") as f:
    json.dump(eval_dataset_filled_c400_40, f, ensure_ascii=False, indent=4)

In [3]:
with open("evaluation/eval_dataset_filled_c400_40.json", "r", encoding="utf-8") as f:
    eval_dataset_filled_c400_40 = json.load(f)

**Evaluation**

In [33]:
from datasets import Dataset
from ragas import evaluate
from langchain_openai import OpenAIEmbeddings
import os
from dotenv import load_dotenv

load_dotenv()  # ensure OPENAI_API_KEY is set

# Prepare your dataset
dataset = Dataset.from_list(eval_dataset_filled_c400_40)
dataset = dataset.rename_columns({
    "question": "user_input",
    "bot_answer": "response",
    "answer": "reference"
})

# Create OpenAI embeddings (CPU only)
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# Evaluate
result = evaluate(dataset, embeddings=embeddings)
print(result)


Evaluating:   0%|          | 0/140 [00:00<?, ?it/s]

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.

KeyboardInterrupt

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.


In [4]:
import os
from datasets import Dataset
from openai import OpenAI
from ragas.llms import llm_factory
from langchain_openai import OpenAIEmbeddings
from ragas import evaluate
import nest_asyncio
nest_asyncio.apply()


In [5]:
# Prepare your dataset
dataset = Dataset.from_list(eval_dataset_filled_c400_40)
dataset = dataset.rename_columns({
    "question": "user_input",
    "bot_answer": "response",
    "answer": "reference"
})


In [14]:
sampled_dataset = dataset.shuffle(seed=42).select(range(10))

In [6]:
# 1️⃣ Create an OpenAI client
#client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [7]:
# 2️⃣ Create evaluator LLM
# "provider" should match your API type ("openai")
llm = llm_factory("gpt-3.5-turbo", provider="openai", client=client)

In [ ]:
import os

os.makedirs("data/embeddings_cache", exist_ok=True)


In [31]:
# 3️⃣ Create embeddings via langchain_openai
embeddings = OpenAIEmbeddings(model="text-embedding-3-large",
                              model_kwargs={"persist": "data/embeddings_cache"})

In [32]:
from ragas.metrics import (
    AnswerRelevancy,
    Faithfulness,
    ContextPrecision,
    ContextRecall,
    AnswerSimilarity
)

# Instantiate metrics
metrics_to_compute = [
    AnswerRelevancy(),
    Faithfulness(),
    ContextPrecision(),
    ContextRecall(),
    AnswerSimilarity()
]

/var/folders/mm/fxsq_1490x9dd2w76tqvt3kr0000gn/T/ipykernel_50753/658222662.py:1: DeprecationWarning: Importing AnswerRelevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import AnswerRelevancy
  from ragas.metrics import (
/var/folders/mm/fxsq_1490x9dd2w76tqvt3kr0000gn/T/ipykernel_50753/658222662.py:1: DeprecationWarning: Importing Faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import Faithfulness
  from ragas.metrics import (
/var/folders/mm/fxsq_1490x9dd2w76tqvt3kr0000gn/T/ipykernel_50753/658222662.py:1: DeprecationWarning: Importing ContextPrecision from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import ContextPrecision
  from ragas.metrics import (
/var/fol

In [33]:
# 4️⃣ Evaluate
result = evaluate(sampled_dataset, embeddings=embeddings, 
                  llm=llm, metrics=metrics_to_compute)

Evaluating:   0%|          | 0/50 [00:00<?, ?it/s]

LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Exception raised in Job[0]: TypeError(Embeddings.create() got an unexpected keyword argument 'persist')
Exception raised in Job[5]: TypeError(Embeddings.create() got an unexpected keyword argument 'persist')
Exception raised in Job[10]: TypeError(Embeddings.create() got an unexpected keyword argument 'persist')
Exception raised in Job[15]: TypeError(Embeddings.create() got an unexpected keyword argument 'persist')
Exception raised in Job[4]: TypeError(AsyncEmbeddings.create() got an unexpected keyword argument 'persist')
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Exception raised in Job[9]: TypeError(AsyncEmbeddings.create() 

In [34]:
print(result)

{'answer_relevancy': nan, 'faithfulness': 0.8006, 'context_precision': 0.9854, 'context_recall': 1.0000, 'answer_similarity': nan}


In [29]:
print(result)

{'answer_relevancy': 0.8382, 'faithfulness': 0.8509, 'context_precision': 0.9842, 'context_recall': 1.0000, 'answer_similarity': 0.9135}


In [ ]:
# Dataset
dataset = Dataset.from_list(eval_dataset_filled)
dataset = dataset.rename_columns({
    "question": "user_input",
    "bot_answer": "response",
    "answer": "reference"
})

# LLM wrapper (for evaluation metrics)
llm = LangchainLLMWrapper(ChatOpenAI(model_name="gpt-3.5-turbo"))

# OpenAI embeddings
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# Evaluate with both embeddings and LLM
result = evaluate(dataset, embeddings=embeddings, llm=llm)
print(result)


In [5]:
pip install --upgrade langchain-openai openai ragas datasets python-dotenv

  Obtaining dependency information for openai from https://files.pythonhosted.org/packages/cc/56/0a89092a453bb2c676d66abee44f863e742b2110d4dbb1dbcca3f7e5fc33/openai-2.21.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.0 MB/s eta 0:00:00m eta 0:00:010:01
  Attempting uninstall: openai
    Found existing installation: openai 2.8.1
    Uninstalling openai-2.8.1:
      Successfully uninstalled openai-2.8.1

[notice] A new release of pip is available: 23.2.1 -> 26.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
dataset.column_names

['user_input', 'reference', 'source', 'response', 'contexts']

In [10]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness
)

# Convert your list-of-dicts to a HuggingFace Dataset
dataset_for_eval = Dataset.from_list(eval_dataset_filled)

# Run RAGAS evaluation
results = evaluate(
    dataset_for_eval,
    metrics=[
        faithfulness,
        answer_relevancy,
        context_recall,
        context_precision,
        answer_correctness
    ],
    prediction_field="bot_answer",   # tell RAGAS which field contains the bot answer
    reference_field="answer",        # tell RAGAS which field is the gold/reference
    contexts_field="contexts"        # field for retrieved contexts
)

print(results)


C:\Users\tzpol\AppData\Local\Temp\ipykernel_10444\4230081972.py:3: DeprecationWarning: Importing faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import faithfulness
  from ragas.metrics import (
C:\Users\tzpol\AppData\Local\Temp\ipykernel_10444\4230081972.py:3: DeprecationWarning: Importing answer_relevancy from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import answer_relevancy
  from ragas.metrics import (
C:\Users\tzpol\AppData\Local\Temp\ipykernel_10444\4230081972.py:3: DeprecationWarning: Importing context_recall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import context_recall
  from ragas.metrics import (
C:\Users\tzpol\AppData\Local\Temp\ipykernel_10444\42

TypeError: evaluate() got an unexpected keyword argument 'prediction_field'

**Pipeline**

In [8]:
def evaluate_rag_semantic(dataset, index, metadata, bm25, 
                            model="gpt-4o-mini",  
                            k=20,top_k=5, 
                            weight_dense=0.6, 
                            weight_sparse=0.4, 
                            rerank=False):
    results = []

    for item in dataset:
        question = item["question"]
        reference = item["answer"]
        source = item.get("source", "unknown")

        # Generate answer
        start_time = time.time()
        generated_answer, retrieved_docs = chatbot.answer(question, index, 
                                                        metadata, bm25,
                                                        model, k, top_k, 
                                                        weight_dense, 
                                                        weight_sparse, 
                                                        rerank)
        latency = time.time() - start_time

        if isinstance(generated_answer, tuple):
            generated_answer = generated_answer[0]

        # Metrics
        retrieval_success = retrieval_success_semantic(retrieved_docs, reference)
        grounding = grounding_score_semantic(generated_answer, retrieved_docs)
        f1 = f1_score(generated_answer, reference)
        conciseness = conciseness_score(generated_answer, reference)
        semantic_sim = semantic_similarity_score(generated_answer, [reference])
        retrieval_recall_val = retrieval_recall(retrieved_docs, reference)
        claim_coverage_val = claim_coverage_score(generated_answer, reference)

        results.append({
            "Question": question,
            "Reference Answer": reference,
            "Source": source,
            "Generated Answer": generated_answer,
            "Retrieval Success (semantic)": retrieval_success,
            "Grounding (semantic)": grounding,
            "F1": f1,
            "Conciseness": conciseness,
            "Semantic Similarity": semantic_sim,
            "retrieval_recall": retrieval_recall_val,
            "claim_coverage_score": claim_coverage_val,
            "Latency (s)": latency
        })

    df = pd.DataFrame(results)
    return df

**Evaluation**

In [9]:
results_1 = {}
for k in [5, 10]:
    results_1[k] = evaluate_rag_semantic(documents, index, metadata, bm25, 
                                        model="gpt-4o-mini",  
                                        k=k,top_k=5, 
                                        weight_dense=0.5, 
                                        weight_sparse=0.5, 
                                        rerank=False)



Assistente: O Alentejo 2030 é um programa que visa promover a competitividade da economia, a sustentabilidade ambiental e a valorização do território e das pessoas na região do Alentejo, no âmbito da Política de Coesão da União Europeia. O programa é constituído por cinco Objetivos Estratégicos que se articulam com as Prioridades e Objetivos Específicos, e tem como objetivo operacionalizar a estratégia de desenvolvimento da região e das suas comunidades intermunicipais. Através do Alentejo 2030, pretende-se transformar vidas, impulsionando a inovação e o crescimento da região, além de disponibilizar Fundos Europeus para apoiar esses objetivos. 

Fonte: alentejo.portugal2030.pt.

Assistente: Alguns dos eventos programados para o Alentejo 2030 incluem:

- 5 de junho de 2025: Como os Fundos Europeus potenciam a Inovação Social
- 12 de junho de 2025: Sessão de apresentação das Plataformas Regionais, integrada no contexto da Estratégia de Desenvolvimento Regional EREI 2030
- 10 de abril de

In [13]:
results_2 = {}
for k in [5, 10]:
    results_2[k] = evaluate_rag_semantic(documents, index, metadata, bm25, 
                                        model="gpt-4o-mini",  
                                        k=k,top_k=5, 
                                        weight_dense=0.5, 
                                        weight_sparse=0.5, 
                                        rerank=True)



Assistente: O Alentejo 2030 é um programa regional que tem como objetivo promover a competitividade da economia, a sustentabilidade ambiental e a valorização do território e das pessoas na região do Alentejo. Aprovado pela Comissão Europeia para o período de programação 2021-2027, o programa visa operacionalizar a estratégia de desenvolvimento da região e as estratégias sub-regionais das Comunidades Intermunicipais, no âmbito da Política de Coesão da União Europeia.

Se precisar de mais informações ou ajuda com outro tema, estou à disposição!

Assistente: Alguns dos eventos programados para o Alentejo 2030 incluem:

1. **Workshop – Prospetivas e Políticas Públicas** - 15 de Dezembro de 2025.
2. **Conferência e Espetáculo | “Inteligência Artificial”** - 27 de Novembro de 2025.
3. **Concerto – “Ecoando a voz do Alentejo”** - 16 de Novembro de 2025, na Vidigueira.
4. **Sessão de apresentação das Plataformas Regionais** - 12 de Junho de 2025.
5. **Como os Fundos Europeus potenciam a Inova

In [21]:
results_3 = {}
for k in [5, 10]:
    results_3[k] = evaluate_rag_semantic(documents, index, metadata, bm25, 
                                        model="gpt-4o-mini",  
                                        k=k,top_k=5, 
                                        weight_dense=0.4, 
                                        weight_sparse=0.6, 
                                        rerank=False)



Assistente: O Alentejo 2030 é um programa regional aprovado pela Comissão Europeia para o período de programação 2021-2027. O seu objetivo é promover o desenvolvimento da região do Alentejo através de investimentos em inovação, sustentabilidade e crescimento económico. O programa visa apoiar diversas áreas, contribuindo para a melhoria da qualidade de vida e a competitividade da região, através da mobilização de fundos europeus.

Se precisar de mais informações ou ajuda com outro tema, estou à disposição!

Assistente: Alguns dos eventos programados para o Alentejo 2030 incluem:

1. **Workshop – Prospetivas e Políticas Públicas** - 15 de Dezembro de 2025.
2. **Conferência e Espetáculo | “Inteligência Artificial”** - 27 de Novembro de 2025.
3. **Concerto – “Ecoando a voz do Alentejo”** - 16 de Novembro de 2025, na Vidigueira.
4. **Sessão de apresentação das Plataformas Regionais** - 12 de Junho de 2025.
5. **Como os Fundos Europeus potenciam a Inovação Social** - 5 de Junho de 2025.

Se

**Results Comparison**

In [14]:
# main vector db (400-80)
results_1[5].describe()

,Retrieval Success (semantic),Grounding (semantic),F1,Conciseness,Semantic Similarity,retrieval_recall,claim_coverage_score,Latency (s)
count,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000
mean,0.779863,0.837757,0.491872,0.778799,0.856668,0.671429,0.880952,2.598972
std,0.094919,0.084623,0.190413,0.389276,0.107099,0.468773,0.306710,1.027622
min,0.486603,0.453830,0.059701,0.425000,0.455210,0.000000,0.000000,1.623515
25%,0.745059,0.812163,0.361047,0.562640,0.823202,0.000000,1.000000,1.963115
50%,0.780513,0.852506,0.476190,0.677419,0.884364,1.000000,1.000000,2.248284
75%,0.845015,0.880531,0.647863,0.842105,0.916822,1.000000,1.000000,2.880953
max,0.907826,0.942501,0.877551,2.178571,0.959933,1.000000,1.000000,6.555275


In [11]:
# main vector db (800-150)
results_1[5].describe()

,Retrieval Success (semantic),Grounding (semantic),F1,Conciseness,Semantic Similarity,retrieval_recall,claim_coverage_score,Latency (s)
count,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000
mean,0.769941,0.828711,0.405759,0.496213,0.841940,0.585714,0.795238,4.659473
std,0.094362,0.087170,0.183803,0.344021,0.113170,0.492379,0.392090,1.622960
min,0.520808,0.477615,0.102564,0.196629,0.471256,0.000000,0.000000,2.032452
25%,0.713674,0.787250,0.278215,0.328321,0.806371,0.000000,1.000000,3.385896
50%,0.782927,0.837019,0.375000,0.415584,0.885896,1.000000,1.000000,4.783664
75%,0.832713,0.877084,0.546414,0.556901,0.917187,1.000000,1.000000,5.726114
max,0.906531,0.950258,0.857143,2.103448,0.954457,1.000000,1.000000,9.442897


In [9]:
# main vector db (500-100)
results_1[5].describe()

,Retrieval Success (semantic),Grounding (semantic),F1,Conciseness,Semantic Similarity,retrieval_recall,claim_coverage_score,Latency (s)
count,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000
mean,0.789211,0.841084,0.502815,0.749259,0.854656,0.700000,0.909524,3.470520
std,0.090409,0.082945,0.207236,0.444660,0.111916,0.456972,0.266123,0.959692
min,0.492678,0.444754,0.108108,0.373626,0.433706,0.000000,0.000000,1.827905
25%,0.764128,0.822422,0.368119,0.515686,0.818578,0.000000,1.000000,2.662459
50%,0.794090,0.846303,0.472222,0.666667,0.870682,1.000000,1.000000,3.356894
75%,0.846317,0.885267,0.655004,0.817036,0.926184,1.000000,1.000000,4.022702
max,0.908167,0.942146,0.870968,3.050000,0.967469,1.000000,1.000000,6.076466


In [10]:
# main vector db (250-50)
results_1[5].describe()

,Retrieval Success (semantic),Grounding (semantic),F1,Conciseness,Semantic Similarity,retrieval_recall,claim_coverage_score,Latency (s)
count,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000
mean,0.789211,0.841084,0.502815,0.749259,0.854656,0.700000,0.909524,3.470520
std,0.090409,0.082945,0.207236,0.444660,0.111916,0.456972,0.266123,0.959692
min,0.492678,0.444754,0.108108,0.373626,0.433706,0.000000,0.000000,1.827905
25%,0.764128,0.822422,0.368119,0.515686,0.818578,0.000000,1.000000,2.662459
50%,0.794090,0.846303,0.472222,0.666667,0.870682,1.000000,1.000000,3.356894
75%,0.846317,0.885267,0.655004,0.817036,0.926184,1.000000,1.000000,4.022702
max,0.908167,0.942146,0.870968,3.050000,0.967469,1.000000,1.000000,6.076466


In [22]:
results_3[5].describe()

,Retrieval Success (semantic),Grounding (semantic),F1,Conciseness,Semantic Similarity,retrieval_recall,claim_coverage_score,Latency (s)
count,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000
mean,0.777910,0.761203,0.422151,0.547518,0.784946,0.671429,0.700000,2.385731
std,0.094534,0.072665,0.152257,0.295830,0.087397,0.468773,0.456972,0.485718
min,0.486144,0.460940,0.081301,0.265487,0.460157,0.000000,0.000000,1.534281
25%,0.744646,0.735717,0.322686,0.413333,0.753969,0.000000,0.000000,1.965801
50%,0.777766,0.776102,0.395604,0.500000,0.804592,1.000000,1.000000,2.237435
75%,0.845094,0.795891,0.535609,0.571429,0.829653,1.000000,1.000000,2.715360
max,0.908160,0.905292,0.735849,2.103448,0.920520,1.000000,1.000000,3.334891


In [16]:
results_1[10].describe()

,Retrieval Success (semantic),Grounding (semantic),F1,Conciseness,Semantic Similarity,retrieval_recall,claim_coverage_score,Latency (s)
count,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000
mean,0.779582,0.832915,0.494076,0.806518,0.849237,0.671429,0.871429,2.360991
std,0.094285,0.087373,0.210790,0.471799,0.116716,0.468773,0.328608,1.247293
min,0.486723,0.443970,0.111111,0.425000,0.433559,0.000000,0.000000,1.433127
25%,0.752425,0.804595,0.340718,0.586016,0.810477,0.000000,1.000000,1.957963
50%,0.780532,0.850129,0.467532,0.693548,0.874407,1.000000,1.000000,2.137947
75%,0.845071,0.882429,0.674159,0.847319,0.923311,1.000000,1.000000,2.352550
max,0.910047,0.942134,0.916667,3.050000,0.964728,1.000000,1.000000,9.125016


In [20]:
results_3[10].describe()

,Retrieval Success (semantic),Grounding (semantic),F1,Conciseness,Semantic Similarity,retrieval_recall,claim_coverage_score,Latency (s)
count,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000
mean,0.781143,0.765255,0.401791,0.528580,0.784260,0.671429,0.700000,2.413206
std,0.094558,0.073652,0.153566,0.317702,0.096619,0.468773,0.456972,0.572270
min,0.486210,0.460938,0.098361,0.252101,0.435490,0.000000,0.000000,1.438702
25%,0.751766,0.736603,0.304362,0.396896,0.751269,0.000000,0.000000,2.038007
50%,0.792334,0.774692,0.394161,0.452174,0.811746,1.000000,1.000000,2.349327
75%,0.844790,0.811717,0.510979,0.557551,0.837001,1.000000,1.000000,2.681686
max,0.908122,0.905320,0.735849,2.103448,0.919313,1.000000,1.000000,3.653453


In [11]:
results_1[15].describe()

,Retrieval Success (semantic),Grounding (semantic),F1,Conciseness,Semantic Similarity,retrieval_recall,claim_coverage_score,Latency (s)
count,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000
mean,0.775133,0.819762,0.456050,0.653927,0.831266,0.642857,0.795238,2.436645
std,0.097962,0.086241,0.171538,0.307099,0.113956,0.478531,0.392090,0.484050
min,0.486634,0.455966,0.078947,0.361702,0.466747,0.000000,0.000000,1.632949
25%,0.743929,0.786691,0.342216,0.507087,0.792915,0.000000,1.000000,2.048267
50%,0.780114,0.824143,0.424242,0.603175,0.862946,1.000000,1.000000,2.354473
75%,0.845530,0.876780,0.593594,0.677778,0.908070,1.000000,1.000000,2.711784
max,0.908114,0.941470,0.772277,2.103448,0.958973,1.000000,1.000000,3.499860


In [23]:
results_1[20].describe().loc["mean"]

Retrieval Success (semantic)    0.799489
Grounding (semantic)            0.819193
F1                              0.405858
Conciseness                     0.526603
Semantic Similarity             0.839202
Latency (s)                     2.204843
Name: mean, dtype: float64

In [24]:
results_2[20].describe().loc["mean"]

Retrieval Success (semantic)    0.797273
Grounding (semantic)            0.811219
F1                              0.397088
Conciseness                     0.486178
Semantic Similarity             0.819887
Latency (s)                     2.322734
Name: mean, dtype: float64

In [25]:
results_3[20].describe().loc["mean"]

Retrieval Success (semantic)    0.759946
Grounding (semantic)            0.805556
F1                              0.387738
Conciseness                     0.561103
Semantic Similarity             0.798391
Latency (s)                     2.018761
Name: mean, dtype: float64

In [26]:
results_4[20].describe().loc["mean"]

Retrieval Success (semantic)    0.759969
Grounding (semantic)            0.807498
F1                              0.381082
Conciseness                     0.550583
Semantic Similarity             0.796183
Latency (s)                     2.056838
Name: mean, dtype: float64

In [11]:
results[10][results[10]['F1'] < 0.2]

,Question,Reference Answer,Source,Generated Answer,Retrieval Success (semantic),Grounding (semantic),F1,Conciseness,Semantic Similarity,Latency (s)
1,Quais são alguns dos eventos programados para ...,Alguns dos eventos programados incluem um work...,alentejo2030,Um dos eventos destacados para o Alentejo 2030...,0.616848,0.883755,0.190476,0.779661,0.599146,2.872252
27,Quais foram as atualizações principais do Avis...,As atualizações principais do Aviso MPr-2025-1...,lisboa2030,"O Aviso MPr-2025-14, que trata da Internaciona...",0.619705,0.817739,0.144144,0.881356,0.633559,1.859997
32,Quais são os principais eventos relacionados a...,Um dos principais eventos é a Mostra dos Fundo...,portugal2030,Os principais eventos relacionados ao Portugal...,0.422468,0.789208,0.155340,0.430556,0.519059,1.769419


In [9]:
results[10].describe()

,Retrieval Success (semantic),Grounding (semantic),F1,Conciseness,Semantic Similarity,Latency (s)
count,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000
mean,0.781333,0.813265,0.384332,0.549461,0.800557,2.519529
std,0.131128,0.060378,0.153616,0.190794,0.109727,1.152649
min,0.422468,0.693576,0.144144,0.225564,0.519059,1.357077
25%,0.699726,0.775607,0.265968,0.432259,0.755421,1.958232
50%,0.795818,0.817739,0.372549,0.522388,0.825233,2.380162
75%,0.888722,0.866697,0.457329,0.697826,0.890280,2.712718
max,0.979312,0.940852,0.764045,0.936170,0.942435,8.395903


In [22]:
results[10].describe()

,Retrieval Success (semantic),Grounding (semantic),F1,Conciseness,Semantic Similarity,Latency (s)
count,35.000000,35.000000,35.000000,35.000000,35.000000,35.000000
mean,0.791556,0.816152,0.387394,0.503692,0.825551,2.127450
std,0.124246,0.060284,0.151614,0.191956,0.092821,0.632687
min,0.472912,0.719310,0.100000,0.223022,0.591620,1.216619
25%,0.719583,0.765352,0.279165,0.376769,0.790033,1.712345
50%,0.797164,0.806561,0.369748,0.460870,0.837484,1.971969
75%,0.888470,0.867891,0.496295,0.570232,0.894935,2.372237
max,0.979312,0.951010,0.666667,1.000000,0.937505,4.114343


In [13]:
small_k_10.describe().loc["mean"]

Retrieval Success (semantic)    0.733697
Grounding (semantic)            0.816320
F1                              0.474633
Conciseness                     0.861939
Semantic Similarity             0.786850
Latency (s)                     1.784377
Name: mean, dtype: float64

In [14]:
small_k_10_rerank.describe().loc["mean"]

Retrieval Success (semantic)    0.733699
Grounding (semantic)            0.813433
F1                              0.467515
Conciseness                     0.853704
Semantic Similarity             0.784782
Latency (s)                     1.548932
Name: mean, dtype: float64

In [36]:
small_k_5_rerank.describe().to_csv(f'evaluation/small_k_5_rerank.csv')

In [20]:
pd.set_option('display.max_colwidth', None)

## Bot-Based

In [54]:
def eval_chat(question, expected_answer, actual_tuple):
    """
    Returns a similarity score 0-10
    """
    actual_answer, retrieved_docs = actual_tuple

    # Embedding-based similarity with the reference
    semantic_sim = semantic_similarity_score(actual_answer, [expected_answer])

    # Scale from 0-1 → 0-10
    score = int(round(semantic_sim * 10))
    return score


In [71]:
from tqdm import tqdm
import time
import pandas as pd

def bot_evaluation(dataset, index, metadata, k=5, model="gpt-4o-mini"):
    results = []

    for idx, item in enumerate(tqdm(dataset, desc="Evaluating RAG")):
        question = item["question"]
        expected_answer = item["answer"]
        source = item.get("source", "unknown")

        # --- Generate answer ---
        start_time = time.time()
        generated_answer, retrieved_docs = chatbot.answer(
            question,
            index,
            metadata,
            k=k,
            model=model
        )
        latency = time.time() - start_time

        if isinstance(generated_answer, tuple):
            generated_answer = generated_answer[0]

        # --- Compute BPT score (0-10) ---
        # You can include grounding if you want
        semantic_sim = semantic_similarity_score(generated_answer, [expected_answer])
        grounding = grounding_score_semantic(generated_answer, retrieved_docs)
        # Weighted combination (optional)
        bpt_score = int(round(0.7 * semantic_sim * 10 + 0.3 * grounding * 10))

        results.append({
            "Question_ID": idx,
            "Source": source,
            "Question": question,
            "Expected Answer": expected_answer,
            "Generated Answer": generated_answer,
            "BPT Score": bpt_score,
            "Response Time (s)": latency
        })

    return pd.DataFrame(results)


In [72]:
results = bot_evaluation(documents, index, metadata)

Evaluating RAG:   0%|                                    | 0/30 [00:00<?, ?it/s]


Assistente: O Alentejo 2030 é um programa regional aprovado pela Comissão Europeia para o período de 2021-2027, que visa promover a inovação, a sustentabilidade e o crescimento da região do Alentejo. Através deste programa, a região dispõe de uma dotação global significativa de fundos europeus, destinados a apoiar diversas áreas e iniciativas que contribuam para o desenvolvimento regional. (Fontes: alentejo2030, portugal2030)


Evaluating RAG:   3%|▉                           | 1/30 [00:09<04:33,  9.44s/it]


Assistente: O documento não especifica eventos concretos programados para o Alentejo 2030. No entanto, menciona que haverá um evento que celebra projetos transformadores em Portugal, incluindo iniciativas do Alentejo 2030, com debates, apresentações e a cerimónia de entrega dos Prémios dos Fundos Europeus. Para informações mais detalhadas sobre eventos específicos, recomendo consultar o site oficial do Alentejo 2030. (Fontes: alentejo2030, portugal2030)


Evaluating RAG:   7%|█▊                          | 2/30 [00:15<03:33,  7.62s/it]

...

Evaluating RAG:  93%|█████████████████████████▏ | 28/30 [03:29<00:12,  6.44s/it]


Assistente: O objetivo do projeto CustomNiTi é desenvolver placas e distratores ósseos personalizados que melhoram a eficiência e o conforto no tratamento de fraturas ósseas. Este projeto, cofinanciado pelo Programa COMPETE 2030, visa inovar o setor de dispositivos médicos, proporcionando soluções clínicas eficazes e económicas. (Fontes: portugal2030)


Evaluating RAG:  97%|██████████████████████████ | 29/30 [03:35<00:06,  6.30s/it]


Assistente: Você pode entrar em contato com a Linha dos Fundos para esclarecer dúvidas sobre o Portugal 2030 através dos seguintes meios:

- **Telefone**: (+351) 289 895 232/37 (custo gratuito para chamadas de Portugal)
- **E-mail**: linhadosfundos@linhadosfundos.pt

A Linha dos Fundos está disponível todos os dias úteis, entre as 9h e as 18h. (Fontes: portugal2030)


Evaluating RAG: 100%|███████████████████████████| 30/30 [03:41<00:00,  7.37s/it]


In [73]:
results.describe()

,Question_ID,BPT Score,Response Time (s)
count,30.000000,30.000000,30.000000
mean,14.500000,5.866667,4.013028
std,8.803408,0.899553,1.232095
min,0.000000,4.000000,2.097220
25%,7.250000,5.250000,3.123637
50%,14.500000,6.000000,3.726955
75%,21.750000,6.000000,4.770928
max,29.000000,7.000000,6.723830


In [75]:
results[results['BPT Score'] < 6]

,Question_ID,Source,Question,Expected Answer,Generated Answer,BPT Score,Response Time (s)
1,1,alentejo2030,Quais são alguns dos eventos programados para o Alentejo 2030?,Alguns eventos programados incluem um workshop sobre Prospetivas e Políticas Públicas em 15 de dezembro de 2025 e uma conferência sobre Inteligência Artificial em 27 de novembro de 2025.,"O documento não especifica eventos concretos programados para o Alentejo 2030. No entanto, menciona que haverá um evento que celebra projetos transformadores em Portugal, incluindo iniciativas do Alentejo 2030, com debates, apresentações e a cerimónia de entrega dos Prémios dos Fundos Europeus. Para informações mais detalhadas sobre eventos específicos, recomendo consultar o site oficial do Alentejo 2030. (Fontes: alentejo2030, portugal2030)",4,3.890048
3,3,alentejo2030,Quais são algumas das novas prioridades específicas integradas no Alentejo 2030?,"As novas prioridades incluem a Defesa, Água, Habitação a preços acessíveis e sustentável, e Competências para a descarbonização.","Algumas das novas prioridades específicas integradas no Alentejo 2030 incluem:\n\n1. **Habitação acessível** - Promovendo a fixação populacional e a justiça social.\n2. **Gestão e eficiência da água** - Reforçando a sustentabilidade e resiliência do território.\n\nEstas prioridades visam fortalecer a resposta da região a desafios económicos, sociais, ambientais e territoriais, contribuindo para um desenvolvimento mais sustentável e inclusivo. (Fontes: alentejo2030)",5,4.053267
6,6,algarve2030,Quantas candidaturas foram recebidas até 28 de novembro de 2025 para o ciclo urbano da água?,Foram recebidas 25 candidaturas de Municípios e entidades gestoras do ciclo urbano da água.,"Até 28 de novembro de 2025, foram recebidas 25 candidaturas para o ciclo urbano da água, totalizando um investimento superior a 100 milhões de euros. (Fontes: portugal2030, algarve2030)",5,3.032607
7,7,algarve2030,Qual é o valor total do investimento das candidaturas relacionadas ao ciclo urbano da água?,O investimento total das candidaturas é de mais de 100 milhões de euros.,"O valor total do investimento das candidaturas relacionadas ao ciclo urbano da água é superior a 100 milhões de euros. (Fontes: portugal2030, algarve2030)",5,2.351102
8,8,algarve2030,Quais são algumas das intervenções estratégicas previstas no Programa Algarve 2030?,"As intervenções incluem a redução de perdas nas redes urbanas de água, reabilitação de infraestruturas obsoletas, prevenção e mitigação da intrusão salina, e reutilização de águas residuais tratadas.","Algumas das intervenções estratégicas previstas no Programa Algarve 2030 incluem:\n\n1. **Promoção da competitividade das empresas** - Através de apoio à inovação e digitalização.\n2. **Sustentabilidade ambiental** - Focando na gestão eficiente dos recursos naturais.\n3. **Valorização do território** - Incentivando o desenvolvimento urbano sustentável e a reabilitação de áreas urbanas.\n4. **Qualidade de vida** - Melhorando os serviços públicos e a acessibilidade.\n\nEstas intervenções visam contribuir para um desenvolvimento equilibrado e sustentável da região do Algarve. (Fontes: algarve2030, portugal2030)",4,5.081694
12,12,centro2030,Qual é o valor do financiamento destinado à habitação no âmbito da reprogramação do CENTRO 2030?,O financiamento destinado à habitação (social e a preços acessíveis) é de quase 200 milhões de euros.,"O valor do financiamento destinado à habitação no âmbito da reprogramação do CENTRO 2030 é de quase 200 milhões de euros. Este investimento visa ampliar o parque público de habitação, promovendo a renovação de edifícios devolutos ou vagos. (Fontes: centro2030)",5,3.175320
14,14,centro2030,Quais são algumas das áreas de intervenção do Programa Centro 2030?,"As áreas de intervenção incluem habitação, tecnologias críticas, defesa e segurança, além de garantir a aplicação eficaz e transparente dos fundos europeus.","Algumas das áreas de intervenção do Programa Centro 2030 

You can explicitly say:

Increasing k:

↑ Recall

↓ Precision

↑ Groundedness (usually)

Better chunking:

↑ Precision

↑ Groundedness

Larger chunks:

↑ Recall

↓ Precision

These trade-offs show deep understanding.